In [22]:
import json
import functions as f
import numpy as np

## Unconditional prompt

In [2]:
# gpt3.5 mbti BASELINE
low_tem_gpt3 = f.get_mbti(model='gpt-3.5-turbo')

middle_tem_gpt3 = f.get_mbti(
    model='gpt-3.5-turbo',
    temperature=1
)

high_tem_gpt3 = f.get_mbti(
    model='gpt-3.5-turbo',
    temperature=1.7
)

{"model": "gpt-3.5-turbo", "details": {"E": 38, "I": 25, "S": 19, "N": 62, "T": 35, "F": 34, "J": 51, "P": 15}, "res": "ENTJ", "prompt_tokens": 12678, "input_tokens": 294, "cost": 0.006486000000000004}


In [9]:
unconditional_mbti_gpt3 = {}
unconditional_mbti_gpt3["0.5 temperature unconditional"] = low_tem_gpt3
unconditional_mbti_gpt3["1 temperature unconditional"] = middle_tem_gpt3
unconditional_mbti_gpt3["1.7 temperature unconditional"] = high_tem_gpt3

In [4]:
# gpt4o mbti BASELINE 
low_tem_gpt4o = f.get_mbti(model='gpt-4o')

middle_tem_gpt4o = f.get_mbti(
    model='gpt-4o',
    temperature=1
)

high_tem_gpt4o = f.get_mbti(
    model='gpt-4o',
    temperature=1.7
)

{"model": "gpt-4o", "details": {"E": 22, "I": 41, "S": 21, "N": 60, "T": 35, "F": 34, "J": 60, "P": 6}, "res": "INTJ", "prompt_tokens": 12351, "input_tokens": 279, "cost": 0.06314999999999993}


In [13]:
unconditional_mbti_gpt4o = {}
unconditional_mbti_gpt4o["0.5 temperature unconditional"] = low_tem_gpt4o
unconditional_mbti_gpt4o["1 temperature unconditional"] = middle_tem_gpt4o
unconditional_mbti_gpt4o["1.7 temperature unconditional"] = high_tem_gpt4o

In [2]:
# gpt4o-mini mbti BASELINE
low_tem_gpt4omini = f.get_mbti(model='gpt-4o-mini')

middle_tem_gpt4omini = f.get_mbti(
    model='gpt-4o-mini',
    temperature=1
)

high_tem_gpt4omini = f.get_mbti(
    model='gpt-4o-mini',
    temperature=1.7
)

In [11]:
unconditional_mbti_gpt4omini = {}
unconditional_mbti_gpt4omini["0.5 temperature unconditional"] = low_tem_gpt4omini
unconditional_mbti_gpt4omini["1 temperature unconditional"] = middle_tem_gpt4omini
unconditional_mbti_gpt4omini["1.7 temperature unconditional"] = high_tem_gpt4omini

In [14]:
# Save result of two models using a dict
SAVE_PATH = 'mbti_results/llms_mbti93.json'

llms_mbti = {}
llms_mbti["gpt-3.5"] = unconditional_mbti_gpt3
llms_mbti["gpt-4o"] = unconditional_mbti_gpt4o
llms_mbti["gpt-4o-mini"] = unconditional_mbti_gpt4omini

# Save the dictionary as a JSON object to the file
with open(SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(llms_mbti, json_file, indent=4)  # indent=4 for pretty-printing

In [ ]:
# consistency test, 3 round for each model
# dict structure: round_i -> model_name -> temperature -> model/details
consistency_test = {}

for i in range(3):
    consistency_test[f'round_{i}'] = {}
    
    for model in ['gpt-3.5-turbo', 'gpt-4o', 'gpt-4o-mini']:
        
        middle_tem = f.get_mbti(
            model=model,
            temperature=1, 
            n=1
        )
        high_tem = f.get_mbti(
            model=model,
            temperature=1.7, 
            n=1
        )
        
        consistency_test[f'round_{i}'][model] = {}
        consistency_test[f'round_{i}'][model]["tem 1.0"] = middle_tem
        consistency_test[f'round_{i}'][model]["tem 1.7"] = high_tem

In [18]:
with open("mbti_results/consistency_test.json", 'w', encoding='utf8') as json_file:
    json.dump(consistency_test, json_file, indent=4)  # indent=4 for pretty-printing

In [15]:
# get consistency data for each model 
# dict structure: model_name -> middle/high_tem -> data -> each_dimension

model_consistency = {}
for model in ['gpt-3.5-turbo', 'gpt-4o', 'gpt-4o-mini']: # calculate accuracy for each model
    acc_high_tem = {}
    acc_middle_tem = {}
    model_consistency[model] = {} 
    model_consistency[model]["middle_tem"] = {}
    model_consistency[model]["high_tem"] = {}


    
    for i in range(3): # gather results of three round
        details_middel = consistency_test[f'round_{i}'][model]["tem 1.0"]["details"]
        for key in details_middel.keys():
            if i == 0: 
                acc_middle_tem[key] = [details_middel[key]]
            else: 
                acc_middle_tem[key].append(details_middel[key])
                
        details_high = consistency_test[f'round_{i}'][model]["tem 1.7"]["details"]
        for key in details_high.keys():
            if i == 0: 
                acc_high_tem[key] = [details_high[key]]
            else: 
                acc_high_tem[key].append(details_high[key])
    
    model_consistency[model]["middle_tem"]["data"] = acc_middle_tem
    model_consistency[model]["high_tem"]["data"] = acc_high_tem       
    
# calculate standered deviation
for model in model_consistency.keys():
    for tem in ["middle_tem", "high_tem"]:
        model_tem = model_consistency[model][tem]
        model_tem["standered deviation"] = {}
                
        for dim in model_tem["data"].keys():
            model_tem["standered deviation"][dim] = np.std(model_tem["data"][dim])

In [42]:
with open("mbti_results/consistency_model.json", 'w', encoding='utf8') as json_file:
    json.dump(model_consistency, json_file, indent=4)  # indent=4 for pretty-printing

In [41]:
model_consistency

{'gpt-3.5-turbo': {'middle_tem': {'data': {'E': [14, 18, 14],
    'I': [7, 3, 7],
    'S': [9, 9, 10],
    'N': [18, 18, 17],
    'T': [13, 16, 16],
    'F': [10, 7, 7],
    'J': [19, 19, 19],
    'P': [3, 3, 3]},
   'standered deviation': {'E': 1.8856180831641267,
    'I': 1.8856180831641267,
    'S': 0.4714045207910317,
    'N': 0.4714045207910317,
    'T': 1.4142135623730951,
    'F': 1.4142135623730951,
    'J': 0.0,
    'P': 0.0}},
  'high_tem': {'data': {'E': [14, 13, 15],
    'I': [7, 8, 6],
    'S': [8, 11, 7],
    'N': [19, 16, 20],
    'T': [17, 10, 17],
    'F': [6, 13, 6],
    'J': [15, 18, 20],
    'P': [7, 4, 2]},
   'standered deviation': {'E': 0.816496580927726,
    'I': 0.816496580927726,
    'S': 1.699673171197595,
    'N': 1.699673171197595,
    'T': 3.2998316455372216,
    'F': 3.2998316455372216,
    'J': 2.0548046676563256,
    'P': 2.0548046676563256}}},
 'gpt-4o': {'middle_tem': {'data': {'E': [8, 8, 7],
    'I': [13, 13, 14],
    'S': [8, 7, 6],
    'N': [19, 2

## Conditional prompt (change LLMs' mbti via prompt)

In [22]:
personalities = json.load(
    open('mbti_types/personality_traits.json', 'r', encoding='utf8')
).keys()

personality_traits = json.load(
    open('mbti_types/personality_traits.json', 'r', encoding='utf8')
)

### gpt 3.5


In [61]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
simple_35_condition_mbti = {}

for p in personalities:

    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = f.get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    
    simple_35_condition_mbti[p] = result

# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
complex_35_condition_mbti = {}

for p in personality_traits.keys():

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = f.get_mbti(
        model='gpt-3.5-turbo', 
        add_sys_prompt=condi_prompt
    )
    
    complex_35_condition_mbti[p] = result

### gpt 4o


In [ ]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
simple_4o_condition_mbti = {}

for p in personalities:
    
    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = f.get_mbti(
        model='gpt-4o', 
        add_sys_prompt=condi_prompt
    )
    
    simple_4o_condition_mbti[p] = result
    
# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
complex_4o_condition_mbti = {}

for p in personality_traits.keys():

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = f.get_mbti(
        model='gpt-4o', 
        add_sys_prompt=condi_prompt
    )
    
    complex_4o_condition_mbti[p] = result


### gpt 4o-mini

In [25]:
# Simple Personality-Conditioned Prompting, 1 loop for 16 personalities
simple_4omini_condition_mbti = {}

for p in personalities:
    
    condi_prompt = (
        'Your personality is'
        + p
    )
    
    result = f.get_mbti(
        model='gpt-4o', 
        add_sys_prompt=condi_prompt
    )
    
    simple_4omini_condition_mbti[p] = result
    
# Complex Personality-Conditioned Prompting, 1 loop for 16 personalities
complex_4omini_condition_mbti = {}

for p in personality_traits.keys():

    condi_prompt = (
        f'You are a human with the following personality type:{p}.'
        + f'Your traits are the following:{personality_traits[p]["traits"]}'
    )
    
    result = f.get_mbti(
        model='gpt-4o', 
        add_sys_prompt=condi_prompt
    )
    
    complex_4omini_condition_mbti[p] = result

In [76]:
# save to file
CONDI_SAVE_PATH = 'mbti_results/condition_mbti93.json'

condition_mbti = {}
condition_mbti['simple_35_condition'] = simple_35_condition_mbti
condition_mbti['complex_35_condition'] = complex_35_condition_mbti
condition_mbti['simple_4o_condition'] = simple_4o_condition_mbti
condition_mbti['complex_4o_condition'] = complex_4o_condition_mbti
condition_mbti['simple_4omini_condition'] = simple_4omini_condition_mbti
condition_mbti['complex_4omini_condition'] = complex_4omini_condition_mbti

with open(CONDI_SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(condition_mbti, json_file, indent=4)

In [27]:
# calculate the accuracy
condition_results = json.load(
    open('mbti_results/condition_mbti93.json', 'r', encoding='utf8')
)

accuracy_rate = {}

for r in condition_results.keys(): # r: one of the four conditional mbti results
    temp_acc_num = 0
    temp_dim = [0, 0, 0, 0]
    dim = ['I/E', 'N/S', 'F/T', 'P/J']
    
    for mbti_type in condition_results[r].keys(): # mbti_type: one of 16 MBTI personalities
        res_mbti = condition_results[r][mbti_type]['res'] 
        
        if mbti_type == res_mbti: # total accuracy
            temp_acc_num += 1 
        
        for n in range(4):
            if mbti_type[n] == res_mbti[n]: # accuracy of certain dimesion
                temp_dim[n] += 1  

    accuracy_rate[r] = {} # aggregate dict
    accuracy_rate[r]['total'] = temp_acc_num / 16
    
    for d in range(4):
        accuracy_rate[r][dim[d]] = temp_dim[d] / 16
        
    # print(r + ' done!')


In [29]:
ACC_SAVE_PATH = 'mbti_results/condition_accuracy.json'

with open(ACC_SAVE_PATH, 'w', encoding='utf8') as json_file:
    json.dump(accuracy_rate, json_file, indent=4)